# IMDB 영화 리뷰 감성분석 

In [ ]:
import numpy as np 
import pandas as pd 


df = pd.read_csv('/content/labeledTrainData.tsv', sep =  '\t', quoting = 3 ) # tsv 파일은 tab으로 구분됨, quoting  위에서 세 줄은 해설이라 넘김 

In [ ]:
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [ ]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

# 1. 전처리

In [ ]:
# <br /> 태그는 공백으로 변환 
df['review'] = df.review.str.replace('<br />', ' ')

# 구둣점, 숫자 제거 --> 영어 이외의 문자는 공백으로 변환 
import re 
df['review'] = df.review.apply(lambda x : re.sub('[^A-Za-z]', ' ', x))

In [ ]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

# 2. 학습 및 테스트 데이터 분리 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.review, df.sentiment, test_size = 0.2, 
                                                    stratify = df.sentiment, random_state = 2021 )

# 3. 텍스트 변환 

- CountVectorizer, ngram_range = (1,1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# 이렇게 변환하면 안됨!! --> 동일한 데이터셋이 아니기 때문에 
cvect1 = CountVectorizer(stop_words = 'english')
X_train_cv1 = cvect1.fit_transform(X_train)
X_test_cv1 = cvect1.fit_transform(X_test)

In [ ]:
X_train_cv1.shape, X_test_cv1.shape

((20000, 66971), (5000, 37187))

In [ ]:
# train, test data set에 있는 동일한 단어에 대해서 동일한 값으로 변환하기 위해 아래처럼 해줘야 함 
cvect1 = CountVectorizer(stop_words = 'english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)
X_train_cv1.shape, X_test_cv1.shape

((20000, 66971), (5000, 66971))

In [ ]:
cvect1 = CountVectorizer(stop_words = 'english', ngram_range=(1,2))
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)
X_train_cv1.shape, X_test_cv1.shape

((20000, 1461350), (5000, 1461350))

# 4. 로지스틱 회귀모델로 학습(훈련)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# 객체 생성 및 훈련 
lr1 = LogisticRegression(max_iter=500)
%time lr1.fit(X_train_cv1, y_train)

CPU times: user 7.16 s, sys: 8.69 s, total: 15.9 s
Wall time: 8.15 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# 테스트 데이터에 대해 예측 - 테스트 데이터의 X만 필요 
pred1 = lr1.predict(X_test_cv1)

# 5. 평가 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
score1 = accuracy_score(y_test,pred1)
score1

0.8832

In [ ]:
# 객체 생성 및 훈련 
lr1 = LogisticRegression(max_iter=500)
%time lr1.fit(X_train_cv1, y_train)

# 테스트 데이터에 대해 예측 - 테스트 데이터의 X만 필요 
pred1 = lr1.predict(X_test_cv1)

score1 = accuracy_score(y_test,pred1)
score1

CPU times: user 59.4 s, sys: 39.4 s, total: 1min 38s
Wall time: 52.1 s


0.8914